
download dataset online:
Dataset 1: OPPORTUNITY dataset
https://archive.ics.uci.edu/ml/datasets/OPPORTUNITY +Activity+Recognition



dependency:
!pip install wget

In [1]:
#import urllib
import json
import wget
import os
import pandas as pd
import re
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from random import random
import zipfile

In [2]:
cfg_update = 1
if cfg_update == 1:
    cfg = {}
    cfg['OPPORTUNITY_DATASET'] = {}
    cfg['OPPORTUNITY_DATASET']['Download_URL'] = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip'
    cfg['OPPORTUNITY_DATASET']['Download_Destination'] = 'Dataset/OpportunityUCIDataset'
    cfg['OPPORTUNITY_DATASET']['dataset_folder'] = 'Dataset/OpportunityUCIDataset/OpportunityUCIDataset/dataset'
    cfg['OPPORTUNITY_DATASET']['Normalization_parameter'] = 'Dataset/OPPORTUNITY_norm_parameter.csv'
    
    cfg['SKODA_DATASET'] = {}
    cfg['SKODA_DATASET']['Download_URL'] = 'http://har-dataset.org/lib/exe/fetch.php?media=wiki:dataset:skodaminicp:skodaminicp_2015_08.zip'
    cfg['SKODA_DATASET']['Download_Destination'] = 'Dataset/SKODA_DATASET'
    
    
    
    
    
    
    
    #data = json.dumps(data, indent=4)
    #print(data)
    with open('cfg.json', 'w') as outfile:
        json.dump(cfg, outfile)
        
        
        
        
        

In [3]:
def import_cfg_file(cfg_file_path):
    with open(cfg_file_path) as json_file:
        cfg = json.load(json_file)
    return cfg

def download_dataset(Source_URL, Destination):
    if not os.path.isfile(Destination + '.zip'):
        wget.download(Source_URL, Destination + '.zip')
    with zipfile.ZipFile(Destination + '.zip', 'r') as zip_ref:
        zip_ref.extractall(Destination)
    os.remove(Destination + '.zip')
    
    
def OPPORTUNITY_Data_Loader(OPPORTUNITY_DATASET_FILE_LIST):
    df_OPPORTUNITY_DATASET_All = ''
    for file_name in OPPORTUNITY_DATASET_FILE_LIST:
        if file_name.endswith(".dat"):
            df_OPPORTUNITY_DATASET_current = pd.read_csv(OPPORTUNITY_DATASET_dataset_folder + '/' + file_name, sep = ' ', header = None)
            df_OPPORTUNITY_DATASET_current.columns = OPPORTUNITY_DATASET_column_names
            df_OPPORTUNITY_DATASET_current = df_OPPORTUNITY_DATASET_current.interpolate()
            df_OPPORTUNITY_DATASET_current.replace(np.nan, 0.0, inplace=True)
            df_OPPORTUNITY_DATASET_current['file_name'] = file_name
            if type(df_OPPORTUNITY_DATASET_All) != pd.core.frame.DataFrame:
                df_OPPORTUNITY_DATASET_All = df_OPPORTUNITY_DATASET_current
            else:
                df_OPPORTUNITY_DATASET_All = pd.concat([df_OPPORTUNITY_DATASET_All, df_OPPORTUNITY_DATASET_current])
    return df_OPPORTUNITY_DATASET_All

In [4]:
#import cfg file:

cfg_file_path = 'cfg.json'
cfg = import_cfg_file(cfg_file_path)

URL_OPPORTUNITY_DATASET = cfg['OPPORTUNITY_DATASET']['Download_URL']
Download_Destination_OPPORTUNITY_DATASET = cfg['OPPORTUNITY_DATASET']['Download_Destination']

URL_SKODA_DATASET = cfg['SKODA_DATASET']['Download_URL']
Download_Destination_SKODA_DATASET = cfg['SKODA_DATASET']['Download_Destination']

OPPORTUNITY_DATASET_dataset_folder = cfg['OPPORTUNITY_DATASET']['dataset_folder']

OPPORTUNITY_DATASET_Normalization_parameter = cfg['OPPORTUNITY_DATASET']['Normalization_parameter']

#download data: only if Dataset is not downloaded yet
"""
download_dataset(URL_OPPORTUNITY_DATASET, Download_Destination_OPPORTUNITY_DATASET)
download_dataset(URL_SKODA_DATASET, Download_Destination_SKODA_DATASET)
"""


'\ndownload_dataset(URL_OPPORTUNITY_DATASET, Download_Destination_OPPORTUNITY_DATASET)\ndownload_dataset(URL_SKODA_DATASET, Download_Destination_SKODA_DATASET)\n'

In [5]:
OPPORTUNITY_DATASET_FILE_LIST = os.listdir(OPPORTUNITY_DATASET_dataset_folder)
OPPORTUNITY_DATASET_FILE_LIST

OPPORTUNITY_DATASET_FILE_LIST_Training = []
OPPORTUNITY_DATASET_FILE_LIST_Validation = []
OPPORTUNITY_DATASET_FILE_LIST_Testing = []


for file_name in OPPORTUNITY_DATASET_FILE_LIST:
    if file_name.endswith(".dat"):
        if 'S4' in file_name:
            continue
        if 'S1' in file_name:
            OPPORTUNITY_DATASET_FILE_LIST_Training.append(file_name)
        elif 'ADL3' in file_name:
            OPPORTUNITY_DATASET_FILE_LIST_Validation.append(file_name)
        elif ('ADL4' in file_name) or ('ADL5' in file_name):
            OPPORTUNITY_DATASET_FILE_LIST_Testing.append(file_name)
        else:
            OPPORTUNITY_DATASET_FILE_LIST_Training.append(file_name)

            
OPPORTUNITY_DATASET_FILE_LIST_Training
OPPORTUNITY_DATASET_FILE_LIST_Validation
OPPORTUNITY_DATASET_FILE_LIST_Testing

['S2-ADL4.dat', 'S2-ADL5.dat', 'S3-ADL4.dat', 'S3-ADL5.dat']

In [6]:
#load column names
with open(OPPORTUNITY_DATASET_dataset_folder + '/column_names.txt') as txt_file:
    OPPORTUNITY_DATASET_column_name = txt_file.read().splitlines()

column_names = []
for column_name in OPPORTUNITY_DATASET_column_name:
    column_name = re.split('; |: ', column_name)
    if column_name[0] == 'Column':
        column_names.append(column_name[1])
    #print(column_name)
#OPPORTUNITY_DATASET_column_name
OPPORTUNITY_DATASET_column_names = column_names

In [7]:
OPPORTUNITY_column_inclusive = ['InertialMeasurementUnit', 'Locomotion', 'ML_Both_Arms']
OPPORTUNITY_column_exclusive = ['Quaternion']

OPPORTUNITY_column_name_selected = OPPORTUNITY_DATASET_column_names[0: 37]
for column_name in OPPORTUNITY_DATASET_column_names:
    inclusive_index = 0
    exclusive_index = 1
    for keyword_inclusive in OPPORTUNITY_column_inclusive:
        if column_name.find(keyword_inclusive) != -1:
            inclusive_index = 1
            break
    for keyword_exclusive in OPPORTUNITY_column_exclusive:
        if column_name.find(keyword_exclusive) != -1:
            exclusive_index = 0
            break
    if inclusive_index * exclusive_index == 1:
        OPPORTUNITY_column_name_selected.append(column_name)

OPPORTUNITY_column_name_selected.append('file_name')
OPPORTUNITY_column_name_selected

['1 MILLISEC',
 '2 Accelerometer RKN^ accX',
 '3 Accelerometer RKN^ accY',
 '4 Accelerometer RKN^ accZ',
 '5 Accelerometer HIP accX',
 '6 Accelerometer HIP accY',
 '7 Accelerometer HIP accZ',
 '8 Accelerometer LUA^ accX',
 '9 Accelerometer LUA^ accY',
 '10 Accelerometer LUA^ accZ',
 '11 Accelerometer RUA_ accX',
 '12 Accelerometer RUA_ accY',
 '13 Accelerometer RUA_ accZ',
 '14 Accelerometer LH accX',
 '15 Accelerometer LH accY',
 '16 Accelerometer LH accZ',
 '17 Accelerometer BACK accX',
 '18 Accelerometer BACK accY',
 '19 Accelerometer BACK accZ',
 '20 Accelerometer RKN_ accX',
 '21 Accelerometer RKN_ accY',
 '22 Accelerometer RKN_ accZ',
 '23 Accelerometer RWR accX',
 '24 Accelerometer RWR accY',
 '25 Accelerometer RWR accZ',
 '26 Accelerometer RUA^ accX',
 '27 Accelerometer RUA^ accY',
 '28 Accelerometer RUA^ accZ',
 '29 Accelerometer LUA_ accX',
 '30 Accelerometer LUA_ accY',
 '31 Accelerometer LUA_ accZ',
 '32 Accelerometer LWR accX',
 '33 Accelerometer LWR accY',
 '34 Accelerome

In [8]:
OPPORTUNITY_DATASET_column_names

['1 MILLISEC',
 '2 Accelerometer RKN^ accX',
 '3 Accelerometer RKN^ accY',
 '4 Accelerometer RKN^ accZ',
 '5 Accelerometer HIP accX',
 '6 Accelerometer HIP accY',
 '7 Accelerometer HIP accZ',
 '8 Accelerometer LUA^ accX',
 '9 Accelerometer LUA^ accY',
 '10 Accelerometer LUA^ accZ',
 '11 Accelerometer RUA_ accX',
 '12 Accelerometer RUA_ accY',
 '13 Accelerometer RUA_ accZ',
 '14 Accelerometer LH accX',
 '15 Accelerometer LH accY',
 '16 Accelerometer LH accZ',
 '17 Accelerometer BACK accX',
 '18 Accelerometer BACK accY',
 '19 Accelerometer BACK accZ',
 '20 Accelerometer RKN_ accX',
 '21 Accelerometer RKN_ accY',
 '22 Accelerometer RKN_ accZ',
 '23 Accelerometer RWR accX',
 '24 Accelerometer RWR accY',
 '25 Accelerometer RWR accZ',
 '26 Accelerometer RUA^ accX',
 '27 Accelerometer RUA^ accY',
 '28 Accelerometer RUA^ accZ',
 '29 Accelerometer LUA_ accX',
 '30 Accelerometer LUA_ accY',
 '31 Accelerometer LUA_ accZ',
 '32 Accelerometer LWR accX',
 '33 Accelerometer LWR accY',
 '34 Accelerome

In [9]:
#load OPPORTUNITY_DATASET_Normalization_parameter
if os.path.isfile(OPPORTUNITY_DATASET_Normalization_parameter):
    df_OPPORTUNITY_DATASET_All_describe = pd.read_csv(OPPORTUNITY_DATASET_Normalization_parameter, index_col = 0)
else:
    df_OPPORTUNITY_DATASET_All = OPPORTUNITY_Data_Loader(OPPORTUNITY_DATASET_FILE_LIST)

    #df_OPPORTUNITY_DATASET_All.describe()
    df_OPPORTUNITY_DATASET_All_describe = df_OPPORTUNITY_DATASET_All.describe()
    #release memory
    df_OPPORTUNITY_DATASET_All = ''
    df_OPPORTUNITY_DATASET_All_describe.to_csv(OPPORTUNITY_DATASET_Normalization_parameter)
df_OPPORTUNITY_DATASET_All_describe

NORM_MAX_THRESHOLDS = [3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,
                       3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,
                       3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,
                       3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,   3000,
                       3000,   3000,   3000,   10000,  10000,  10000,  1500,   1500,   1500,
                       3000,   3000,   3000,   10000,  10000,  10000,  1500,   1500,   1500,
                       3000,   3000,   3000,   10000,  10000,  10000,  1500,   1500,   1500,
                       3000,   3000,   3000,   10000,  10000,  10000,  1500,   1500,   1500,
                       3000,   3000,   3000,   10000,  10000,  10000,  1500,   1500,   1500,
                       250,    25,     200,    5000,   5000,   5000,   5000,   5000,   5000,
                       10000,  10000,  10000,  10000,  10000,  10000,  250,    250,    25,
                       200,    5000,   5000,   5000,   5000,   5000,   5000,   10000,  10000,
                       10000,  10000,  10000,  10000,  250, ]

NORM_MIN_THRESHOLDS = [-3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,
                       -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,
                       -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,
                       -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,  -3000,
                       -3000,  -3000,  -3000,  -10000, -10000, -10000, -1000,  -1000,  -1000,
                       -3000,  -3000,  -3000,  -10000, -10000, -10000, -1000,  -1000,  -1000,
                       -3000,  -3000,  -3000,  -10000, -10000, -10000, -1000,  -1000,  -1000,
                       -3000,  -3000,  -3000,  -10000, -10000, -10000, -1000,  -1000,  -1000,
                       -3000,  -3000,  -3000,  -10000, -10000, -10000, -1000,  -1000,  -1000,
                       -250,   -100,   -200,   -5000,  -5000,  -5000,  -5000,  -5000,  -5000,
                       -10000, -10000, -10000, -10000, -10000, -10000, -250,   -250,   -100,
                       -200,   -5000,  -5000,  -5000,  -5000,  -5000,  -5000,  -10000, -10000,
                       -10000, -10000, -10000, -10000, -250, ]


df_OPPORTUNITY_DATASET_All_describe.loc[['min'], [col_name for col_name in OPPORTUNITY_column_name_selected[1: -3]]] = NORM_MIN_THRESHOLDS
df_OPPORTUNITY_DATASET_All_describe.loc[['max'], [col_name for col_name in OPPORTUNITY_column_name_selected[1: -3]]] = NORM_MAX_THRESHOLDS

df_OPPORTUNITY_DATASET_All_describe

,1 MILLISEC,2 Accelerometer RKN^ accX,3 Accelerometer RKN^ accY,4 Accelerometer RKN^ accZ,5 Accelerometer HIP accX,6 Accelerometer HIP accY,7 Accelerometer HIP accZ,8 Accelerometer LUA^ accX,9 Accelerometer LUA^ accY,10 Accelerometer LUA^ accZ,...,241 LOCATION TAG4 X,242 LOCATION TAG4 Y,243 LOCATION TAG4 Z,244 Locomotion,245 HL_Activity,246 LL_Left_Arm,247 LL_Left_Arm_Object,248 LL_Right_Arm,249 LL_Right_Arm_Object,250 ML_Both_Arms
count,8.693870e+05,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,...,868896.000000,868896.000000,868896.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000
mean,6.636642e+05,32.335708,828.805159,364.077286,-188.244306,888.038276,121.149150,165.418518,816.527912,273.289180,...,5496.174763,1936.492256,1329.034868,1.631271,63.157020,35.684623,52.681547,171.959039,215.540657,112607.864146
std,4.596940e+05,302.226970,541.533232,444.932641,269.312627,246.693948,287.552993,314.618077,278.291696,328.882027,...,1873.592758,1156.722048,496.526888,1.379007,50.479497,78.929812,116.544989,201.915249,253.107758,181823.905474
min,0.000000e+00,-3000.000000,-3000.000000,-3000.000000,-3000.000000,-3000.000000,-3000.000000,-3000.000000,-3000.000000,-3000.000000,...,0.000000,0.000000,-2352.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.018640e+05,-104.000000,639.000000,53.000000,-375.000000,857.000000,-15.000000,-7.000000,738.000000,58.000000,...,4298.000000,1055.000000,1099.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.037270e+05,22.000000,962.000000,317.000000,-167.000000,947.000000,116.000000,169.000000,891.000000,280.000000,...,5519.000000,1836.000000,1362.000000,1.000000,102.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.190910e+05,170.500000,1031.000000,633.000000,0.000000,997.000000,273.000000,371.000000,974.000000,462.000000,...,6743.000000,2396.000000,1585.000000,2.000000,104.000000,0.000000,0.000000,407.000000,511.000000,404511.000000
max,2.364210e+06,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,...,10128.000000,5931.000000,4122.000000,5.000000,105.000000,213.000000,323.000000,413.000000,523.000000,408512.000000


In [10]:
OPPORTUNITY_DATASET_FILE_LIST_Training
OPPORTUNITY_DATASET_FILE_LIST_Validation
OPPORTUNITY_DATASET_FILE_LIST_Testing


df_OPPORTUNITY_DATASET_Training = OPPORTUNITY_Data_Loader(OPPORTUNITY_DATASET_FILE_LIST_Training)
df_OPPORTUNITY_DATASET_Validation = OPPORTUNITY_Data_Loader(OPPORTUNITY_DATASET_FILE_LIST_Validation)
df_OPPORTUNITY_DATASET_Testing = OPPORTUNITY_Data_Loader(OPPORTUNITY_DATASET_FILE_LIST_Testing)


In [11]:
rr = df_OPPORTUNITY_DATASET_Testing['33 Accelerometer LWR accY'].unique()
for m in rr:
    print(m)

826.0
852.0
821.0
842.0
795.0
851.0
834.0
832.0
838.0
816.0
819.0
854.0
811.0
766.0
800.0
830.0
822.0
864.0
862.0
856.0
789.0
818.0
869.0
881.0
815.0
775.0
791.0
812.0
831.0
883.0
833.0
783.0
829.0
820.0
846.0
792.0
758.0
778.0
837.0
877.0
885.0
850.0
765.0
817.0
865.0
878.0
810.0
784.0
840.0
813.0
876.0
836.0
798.0
843.0
835.0
814.0
825.0
751.0
808.0
774.0
827.0
769.0
780.0
794.0
803.0
754.0
764.0
799.0
692.0
748.0
711.0
718.0
459.0
395.0
288.0
246.0
39.0
-20.0
-134.0
-259.0
-273.0
-341.0
-244.0
-321.0
-352.0
-390.0
-428.0
-533.0
-677.0
-739.0
-881.0
-962.0
-1017.0
-1106.0
-1009.0
-995.0
-988.0
-954.0
-1020.0
-1008.0
-1033.0
-982.0
-971.0
-1000.0
-978.0
-935.0
-942.15
-949.3
-956.45
-963.6
-970.75
-977.9
-985.05
-992.2
-999.35
-1006.5
-1013.65
-1020.8
-1027.95
-1035.1
-1042.25
-1049.4
-1056.55
-1063.7
-1070.85
-1078.0
-893.0
-980.0
-928.0
-862.0
-887.0
-858.0
-839.0
-825.0
-870.0
-871.0
-897.0
-864.0
-856.0
-845.0
-849.0
-817.0
-831.0
-785.0
-714.0
-872.4285714285714
-873.857142857142

1620.0
1649.0
1366.0
1619.0
1510.0
-112.0
-153.0
-200.0
-194.0
-236.0
1648.0
1657.0
1459.0
-183.0
-196.0
-66.0
-226.0
-235.0
101.0
193.0
315.0
-113.0
-219.0
-105.0
313.0
-170.0
114.0
85.0
84.0
333.0
185.0
-56.0
300.0
-186.0
-163.0
1911.0
2024.0
1281.0
1600.0
1406.0
-261.0
-257.0
-263.0
-207.0
-411.0
-498.0
-573.0
-609.0
-597.0
-476.0
-418.0
-156.0
-490.0
-408.0
-395.0
-381.0
-370.0
-376.0
-371.0
-255.0
-441.0
-451.0
-436.0
-503.0
-501.0
-472.0
-534.0
-529.0
-607.0
-610.0
-557.0
-614.0
-603.0
-595.0
-548.0
-545.0
-558.0
-608.0
-580.0
-587.0
-598.0
-635.0
-562.0
-581.0
-633.0
-579.0
-644.0
-585.0
-615.0
-574.0
-632.0
-612.0
-649.0
-671.0
-631.0
-642.0
-689.0
-620.0
-583.0
-98.0
-526.0
-699.0
-807.0
-916.0
-571.0
-386.0
-400.0
-491.0
-575.0
-422.0
-466.0
-461.0
-561.0
-532.0
-464.0
-425.0
-412.0
-530.0
-414.0
-457.0
-396.0
-518.0
-429.0
-452.0
-478.0
-449.0
-416.0
-404.0
-406.0
-423.0
-447.0
-442.0
-514.0
-646.0
-781.0
-378.0
-538.0
-515.0
-508.0
-525.0
-541.0
-523.0
-504.0
-505.0
-531.0


In [12]:
df_OPPORTUNITY_DATASET_Training.describe()

,1 MILLISEC,2 Accelerometer RKN^ accX,3 Accelerometer RKN^ accY,4 Accelerometer RKN^ accZ,5 Accelerometer HIP accX,6 Accelerometer HIP accY,7 Accelerometer HIP accZ,8 Accelerometer LUA^ accX,9 Accelerometer LUA^ accY,10 Accelerometer LUA^ accZ,...,241 LOCATION TAG4 X,242 LOCATION TAG4 Y,243 LOCATION TAG4 Z,244 Locomotion,245 HL_Activity,246 LL_Left_Arm,247 LL_Left_Arm_Object,248 LL_Right_Arm,249 LL_Right_Arm_Object,250 ML_Both_Arms
count,4.970140e+05,497014.000000,497014.000000,497014.000000,497014.000000,497014.000000,497014.000000,497014.000000,497014.000000,497014.000000,...,497014.000000,497014.000000,497014.000000,497014.000000,497014.000000,497014.000000,497014.000000,497014.000000,497014.000000,497014.000000
mean,7.564166e+05,-37.676746,826.891055,374.531324,-232.655048,883.527196,96.858446,100.135228,852.844785,221.045158,...,5514.212294,1894.938921,1329.812456,1.638386,55.113691,28.173289,41.689989,171.021883,214.786509,129779.274855
std,5.130806e+05,292.281261,388.740889,440.959108,253.050839,252.906259,274.738007,305.068453,261.999301,315.106641,...,1795.768197,1150.564352,472.690114,1.327398,51.597643,71.544644,105.886189,201.607423,253.216033,189399.852776
min,0.000000e+00,-3955.000000,-4288.000000,-6519.000000,-2276.000000,-1103.000000,-2672.000000,-3514.000000,-1829.000000,-4187.000000,...,0.000000,0.000000,-1505.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.451300e+05,-145.000000,644.000000,89.000000,-401.000000,858.000000,-25.000000,-71.000000,789.000000,0.000000,...,4409.000000,983.000000,1110.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.902600e+05,-27.000000,956.000000,338.000000,-237.000000,948.000000,91.000000,111.000000,921.000000,234.000000,...,5622.000000,1803.000000,1344.000000,1.000000,101.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.054156e+06,80.000000,1029.000000,621.000000,-60.000000,995.000000,222.000000,269.000000,989.000000,412.000000,...,6688.000000,2390.000000,1557.000000,2.000000,103.000000,0.000000,0.000000,406.000000,512.000000,404517.000000
max,2.364210e+06,5054.000000,4566.000000,4681.000000,1928.000000,3822.000000,4697.000000,4385.000000,3986.000000,5395.000000,...,10008.000000,5895.000000,3770.000000,5.000000,105.000000,213.000000,323.000000,413.000000,523.000000,408512.000000


In [13]:
df_OPPORTUNITY_DATASET_Testing.describe()

,1 MILLISEC,2 Accelerometer RKN^ accX,3 Accelerometer RKN^ accY,4 Accelerometer RKN^ accZ,5 Accelerometer HIP accX,6 Accelerometer HIP accY,7 Accelerometer HIP accZ,8 Accelerometer LUA^ accX,9 Accelerometer LUA^ accY,10 Accelerometer LUA^ accZ,...,241 LOCATION TAG4 X,242 LOCATION TAG4 Y,243 LOCATION TAG4 Z,244 Locomotion,245 HL_Activity,246 LL_Left_Arm,247 LL_Left_Arm_Object,248 LL_Right_Arm,249 LL_Right_Arm_Object,250 ML_Both_Arms
count,1.187500e+05,118750.000000,118750.000000,118750.000000,118750.000000,118750.000000,118750.000000,118750.000000,118750.000000,118750.000000,...,118750.000000,118750.000000,118750.000000,118750.000000,118750.000000,118750.000000,118750.000000,118750.000000,118750.000000,118750.000000
mean,4.987220e+05,58.054366,779.708552,420.738139,-142.735680,869.509309,22.912792,76.668657,786.634968,449.592029,...,4889.658021,1948.529524,1293.430552,1.821869,88.689128,58.396842,86.077549,167.424514,208.472522,67925.123082
std,2.923931e+05,317.346302,412.463287,458.368282,369.378844,249.179066,275.901180,303.728022,281.788632,263.992885,...,2295.982471,1271.544201,601.997115,1.531071,36.328585,94.159144,138.818252,201.596508,251.046376,151572.825876
min,0.000000e+00,-2546.000000,-2114.000000,-3245.000000,-1698.000000,-476.000000,-2115.000000,-3159.000000,-939.000000,-4018.000000,...,0.000000,0.000000,-879.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.473725e+05,-74.000000,484.000000,88.000000,-444.000000,814.000000,-81.000000,-67.000000,686.000000,278.621684,...,3789.000000,1056.000000,1042.000000,1.000000,102.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.947620e+05,56.634256,929.000000,396.000000,-19.000000,947.000000,39.000000,84.000000,857.000000,415.000000,...,5004.000000,1872.000000,1355.000000,1.000000,103.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.421590e+05,172.000000,1030.000000,872.000000,129.000000,1005.000000,147.000000,234.000000,955.000000,638.000000,...,6663.000000,2498.000000,1643.000000,4.000000,105.000000,205.000000,302.000000,411.000000,507.000000,0.000000
max,1.091556e+06,4439.000000,3649.000000,4525.000000,1754.000000,2285.000000,1337.000000,4199.000000,2512.000000,4686.000000,...,9933.000000,5931.000000,3118.000000,5.000000,105.000000,213.000000,323.000000,413.000000,523.000000,408512.000000


In [14]:
df_OPPORTUNITY_DATASET_Validation.describe()

,1 MILLISEC,2 Accelerometer RKN^ accX,3 Accelerometer RKN^ accY,4 Accelerometer RKN^ accZ,5 Accelerometer HIP accX,6 Accelerometer HIP accY,7 Accelerometer HIP accZ,8 Accelerometer LUA^ accX,9 Accelerometer LUA^ accY,10 Accelerometer LUA^ accZ,...,241 LOCATION TAG4 X,242 LOCATION TAG4 Y,243 LOCATION TAG4 Z,244 Locomotion,245 HL_Activity,246 LL_Left_Arm,247 LL_Left_Arm_Object,248 LL_Right_Arm,249 LL_Right_Arm_Object,250 ML_Both_Arms
count,6.094900e+04,60949.000000,60949.000000,60949.000000,60949.000000,60949.000000,60949.000000,60949.000000,60949.000000,60949.000000,...,60949.000000,60949.000000,60949.000000,60949.000000,60949.000000,60949.000000,60949.000000,60949.000000,60949.000000,60949.000000
mean,5.156082e+05,38.253852,793.383230,419.180192,-185.408227,898.413313,108.346929,74.905692,787.610543,457.320686,...,5719.877980,1927.973059,1308.857274,1.749561,88.936340,65.408473,96.166713,193.071749,240.852188,74037.279873
std,3.062256e+05,288.480765,386.970917,461.032971,290.445728,239.431467,269.844540,302.797328,283.509414,270.406234,...,1807.724753,1025.858908,468.081631,1.529990,36.140967,97.573790,143.492573,204.792228,255.500724,156831.026298
min,0.000000e+00,-2872.000000,-859.000000,-3058.000000,-1798.000000,-140.000000,-2025.000000,-3112.000000,-1198.000000,-4024.000000,...,0.000000,0.000000,-2352.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.539310e+05,-70.000000,431.000000,95.000000,-363.000000,865.000000,-8.000000,-59.000000,686.000000,293.054054,...,4273.000000,1254.000000,992.000000,1.000000,102.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.078950e+05,42.000000,942.000000,376.000000,-145.000000,963.000000,119.314168,97.000000,855.000000,415.000000,...,5387.000000,1817.000000,1340.000000,1.000000,103.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.618260e+05,121.000000,1029.648338,882.000000,-14.000000,1010.000000,227.000000,233.000000,958.000000,654.000000,...,6943.000000,2266.000000,1596.000000,2.000000,105.000000,206.000000,303.000000,413.000000,512.000000,0.000000
max,1.141022e+06,2796.000000,3866.000000,4604.000000,1221.000000,2356.000000,1471.000000,4122.000000,2024.000000,5204.000000,...,9822.000000,5838.000000,2923.000000,5.000000,105.000000,213.000000,322.000000,413.000000,522.000000,408512.000000


In [15]:
df_OPPORTUNITY_DATASET_Training['file_name'].describe()

count           497014
unique              12
top       S3-Drill.dat
freq             70928
Name: file_name, dtype: object

In [16]:
df_OPPORTUNITY_DATASET_All_describe['2 Accelerometer RKN^ accX']['count']
df_OPPORTUNITY_DATASET_All_describe

,1 MILLISEC,2 Accelerometer RKN^ accX,3 Accelerometer RKN^ accY,4 Accelerometer RKN^ accZ,5 Accelerometer HIP accX,6 Accelerometer HIP accY,7 Accelerometer HIP accZ,8 Accelerometer LUA^ accX,9 Accelerometer LUA^ accY,10 Accelerometer LUA^ accZ,...,241 LOCATION TAG4 X,242 LOCATION TAG4 Y,243 LOCATION TAG4 Z,244 Locomotion,245 HL_Activity,246 LL_Left_Arm,247 LL_Left_Arm_Object,248 LL_Right_Arm,249 LL_Right_Arm_Object,250 ML_Both_Arms
count,8.693870e+05,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,...,868896.000000,868896.000000,868896.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000,869387.000000
mean,6.636642e+05,32.335708,828.805159,364.077286,-188.244306,888.038276,121.149150,165.418518,816.527912,273.289180,...,5496.174763,1936.492256,1329.034868,1.631271,63.157020,35.684623,52.681547,171.959039,215.540657,112607.864146
std,4.596940e+05,302.226970,541.533232,444.932641,269.312627,246.693948,287.552993,314.618077,278.291696,328.882027,...,1873.592758,1156.722048,496.526888,1.379007,50.479497,78.929812,116.544989,201.915249,253.107758,181823.905474
min,0.000000e+00,-3000.000000,-3000.000000,-3000.000000,-3000.000000,-3000.000000,-3000.000000,-3000.000000,-3000.000000,-3000.000000,...,0.000000,0.000000,-2352.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.018640e+05,-104.000000,639.000000,53.000000,-375.000000,857.000000,-15.000000,-7.000000,738.000000,58.000000,...,4298.000000,1055.000000,1099.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.037270e+05,22.000000,962.000000,317.000000,-167.000000,947.000000,116.000000,169.000000,891.000000,280.000000,...,5519.000000,1836.000000,1362.000000,1.000000,102.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.190910e+05,170.500000,1031.000000,633.000000,0.000000,997.000000,273.000000,371.000000,974.000000,462.000000,...,6743.000000,2396.000000,1585.000000,2.000000,104.000000,0.000000,0.000000,407.000000,511.000000,404511.000000
max,2.364210e+06,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,...,10128.000000,5931.000000,4122.000000,5.000000,105.000000,213.000000,323.000000,413.000000,523.000000,408512.000000


In [17]:
df_OPPORTUNITY_DATASET_Training['file_name']

0         S1-ADL1.dat
1         S1-ADL1.dat
2         S1-ADL1.dat
3         S1-ADL1.dat
4         S1-ADL1.dat
             ...     
70923    S3-Drill.dat
70924    S3-Drill.dat
70925    S3-Drill.dat
70926    S3-Drill.dat
70927    S3-Drill.dat
Name: file_name, Length: 497014, dtype: object

In [18]:


#len(OPPORTUNITY_column_name_selected)
#df_OPPORTUNITY_DATASET_WIP = df_OPPORTUNITY_DATASET[OPPORTUNITY_column_name_selected]

In [19]:
OPPORTUNITY_column_name_selected

['1 MILLISEC',
 '2 Accelerometer RKN^ accX',
 '3 Accelerometer RKN^ accY',
 '4 Accelerometer RKN^ accZ',
 '5 Accelerometer HIP accX',
 '6 Accelerometer HIP accY',
 '7 Accelerometer HIP accZ',
 '8 Accelerometer LUA^ accX',
 '9 Accelerometer LUA^ accY',
 '10 Accelerometer LUA^ accZ',
 '11 Accelerometer RUA_ accX',
 '12 Accelerometer RUA_ accY',
 '13 Accelerometer RUA_ accZ',
 '14 Accelerometer LH accX',
 '15 Accelerometer LH accY',
 '16 Accelerometer LH accZ',
 '17 Accelerometer BACK accX',
 '18 Accelerometer BACK accY',
 '19 Accelerometer BACK accZ',
 '20 Accelerometer RKN_ accX',
 '21 Accelerometer RKN_ accY',
 '22 Accelerometer RKN_ accZ',
 '23 Accelerometer RWR accX',
 '24 Accelerometer RWR accY',
 '25 Accelerometer RWR accZ',
 '26 Accelerometer RUA^ accX',
 '27 Accelerometer RUA^ accY',
 '28 Accelerometer RUA^ accZ',
 '29 Accelerometer LUA_ accX',
 '30 Accelerometer LUA_ accY',
 '31 Accelerometer LUA_ accZ',
 '32 Accelerometer LWR accX',
 '33 Accelerometer LWR accY',
 '34 Accelerome

In [20]:
def OPPORTUNITY_Normalization(df_OPPORTUNITY_DATASET, df_OPPORTUNITY_DS_All_describe, OPPORTUNITY_col_name_selected):
    for column_name in OPPORTUNITY_col_name_selected:
        if column_name in ['1 MILLISEC', '244 Locomotion', '250 ML_Both_Arms', 'file_name']:
            pass
        else:
            df_OPPORTUNITY_DATASET[column_name] = (df_OPPORTUNITY_DATASET[column_name] - df_OPPORTUNITY_DS_All_describe[column_name]['min']) / (df_OPPORTUNITY_DS_All_describe[column_name]['max'] - df_OPPORTUNITY_DS_All_describe[column_name]['min'])
            df_OPPORTUNITY_DATASET[column_name].loc[df_OPPORTUNITY_DATASET[column_name] > 1] = 1
            df_OPPORTUNITY_DATASET[column_name].loc[df_OPPORTUNITY_DATASET[column_name] < 0] = 0
            
    df_OPPORTUNITY_DATASET = df_OPPORTUNITY_DATASET[OPPORTUNITY_col_name_selected]
    return df_OPPORTUNITY_DATASET
    
    
df_OPPORTUNITY_DATASET_Training_WIP = OPPORTUNITY_Normalization(df_OPPORTUNITY_DATASET_Training, df_OPPORTUNITY_DATASET_All_describe, OPPORTUNITY_column_name_selected)
df_OPPORTUNITY_DATASET_Testing_WIP = OPPORTUNITY_Normalization(df_OPPORTUNITY_DATASET_Testing, df_OPPORTUNITY_DATASET_All_describe, OPPORTUNITY_column_name_selected)
df_OPPORTUNITY_DATASET_Validation_WIP = OPPORTUNITY_Normalization(df_OPPORTUNITY_DATASET_Validation, df_OPPORTUNITY_DATASET_All_describe, OPPORTUNITY_column_name_selected)


Classifications = df_OPPORTUNITY_DATASET_Training_WIP['250 ML_Both_Arms'].unique()
dict_18_classification = {}
class_index = 0
for Classitication in Classifications:
    dict_18_classification[Classitication] = class_index
    class_index += 1

df_OPPORTUNITY_DATASET_Training_WIP['250 ML_Both_Arms'] = df_OPPORTUNITY_DATASET_Training_WIP['250 ML_Both_Arms'].apply(lambda  x: dict_18_classification[x])
df_OPPORTUNITY_DATASET_Testing_WIP['250 ML_Both_Arms'] = df_OPPORTUNITY_DATASET_Testing_WIP['250 ML_Both_Arms'].apply(lambda  x: dict_18_classification[x])
df_OPPORTUNITY_DATASET_Validation_WIP['250 ML_Both_Arms'] = df_OPPORTUNITY_DATASET_Validation_WIP['250 ML_Both_Arms'].apply(lambda  x: dict_18_classification[x])

df_OPPORTUNITY_DATASET_Validation_WIP

D:\Software\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-20-08237c4d2d39>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_OPPORTUNITY_DATASET_Training_WIP['250 ML_Both_Arms'] = df_OPPORTUNITY_DATASET_Training_WIP['250 ML_Both_Arms'].apply(lambda  x: dict_18_classification[x])
<ipython-input-20-08237c4d2d39>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,1 MILLISEC,2 Accelerometer RKN^ accX,3 Accelerometer RKN^ accY,4 Accelerometer RKN^ accZ,5 Accelerometer HIP accX,6 Accelerometer HIP accY,7 Accelerometer HIP accZ,8 Accelerometer LUA^ accX,9 Accelerometer LUA^ accY,10 Accelerometer LUA^ accZ,...,128 InertialMeasurementUnit R-SHOE AngVelBodyFrameX,129 InertialMeasurementUnit R-SHOE AngVelBodyFrameY,130 InertialMeasurementUnit R-SHOE AngVelBodyFrameZ,131 InertialMeasurementUnit R-SHOE AngVelNavFrameX,132 InertialMeasurementUnit R-SHOE AngVelNavFrameY,133 InertialMeasurementUnit R-SHOE AngVelNavFrameZ,134 InertialMeasurementUnit R-SHOE Compass,244 Locomotion,250 ML_Both_Arms,file_name
0,0,0.491000,0.678000,0.490000,0.490333,0.668333,0.506000,0.528833,0.656667,0.552333,...,0.50000,0.50080,0.50015,0.50080,0.50000,0.50015,0.778,0,0,S2-ADL3.dat
1,33,0.488167,0.674167,0.488833,0.490500,0.676167,0.486000,0.517333,0.658333,0.550833,...,0.49970,0.50215,0.50195,0.50215,0.50030,0.50195,0.780,0,0,S2-ADL3.dat
2,67,0.490000,0.668500,0.480667,0.496833,0.674833,0.492833,0.505667,0.666833,0.554500,...,0.49975,0.50210,0.50185,0.50210,0.50025,0.50185,0.780,0,0,S2-ADL3.dat
3,100,0.486833,0.668500,0.475500,0.495000,0.671500,0.508500,0.508333,0.670167,0.552167,...,0.49990,0.50205,0.50205,0.50205,0.50010,0.50205,0.780,0,0,S2-ADL3.dat
4,133,0.493500,0.674167,0.485500,0.496667,0.665333,0.500833,0.519167,0.662667,0.555667,...,0.49935,0.50205,0.50050,0.50205,0.50065,0.50050,0.780,0,0,S2-ADL3.dat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26712,890391,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.49905,0.50125,0.49790,0.50125,0.50095,0.49790,0.534,0,0,S3-ADL3.dat
26713,890424,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.49895,0.49910,0.49895,0.49910,0.50105,0.49895,0.532,0,0,S3-ADL3.dat
26714,890458,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.50120,0.49940,0.50065,0.49940,0.49880,0.50065,0.532,0,0,S3-ADL3.dat
26715,890491,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.50005,0.49995,0.50160,0.49995,0.49995,0.50160,0.536,0,0,S3-ADL3.dat


In [21]:
df_OPPORTUNITY_DATASET_Training_WIP['1 MILLISEC'].count()
df_OPPORTUNITY_DATASET_Training_WIP['file_name'].unique()

array(['S1-ADL1.dat', 'S1-ADL2.dat', 'S1-ADL3.dat', 'S1-ADL4.dat',
       'S1-ADL5.dat', 'S1-Drill.dat', 'S2-ADL1.dat', 'S2-ADL2.dat',
       'S2-Drill.dat', 'S3-ADL1.dat', 'S3-ADL2.dat', 'S3-Drill.dat'],
      dtype=object)

In [22]:

SILDE_WINDOW = 24
SILDE_WINDOW_STEP = 12


def data_generator(df_OPPORTUNITY_DATASET, SILDE_WINDOW, SILDE_WINDOW_STEP, OPPORTUNITY_column_name_selected):
    file_no = 0
    for file_name in df_OPPORTUNITY_DATASET['file_name'].unique():
        file_no += 1
        print(file_no)
        df_OPPORTUNITY_DATASET_WIP = df_OPPORTUNITY_DATASET.loc[df_OPPORTUNITY_DATASET['file_name'] == file_name]
        df_OPPORTUNITY_DATASET_WIP = df_OPPORTUNITY_DATASET_WIP[OPPORTUNITY_column_name_selected[1: ]]
        row_no_start = 0
        
        
        while (row_no_start + SILDE_WINDOW - 1) <= df_OPPORTUNITY_DATASET_WIP['2 Accelerometer RKN^ accX'].count():
            row_no_end = row_no_start + SILDE_WINDOW
            
            df_input = df_OPPORTUNITY_DATASET_WIP[OPPORTUNITY_column_name_selected[1: -3]][row_no_start: row_no_end]
            current_input = torch.tensor([df_input.values])
            
            df_output_Locomotion = df_OPPORTUNITY_DATASET_WIP['244 Locomotion'][row_no_end - 1]
            
            current_output_Locomotion = torch.tensor([df_output_Locomotion])
            #print(current_output_Locomotion)
            df_output_ML_Both_Arms = df_OPPORTUNITY_DATASET_WIP['250 ML_Both_Arms'][row_no_end - 1]
            current_output_ML_Both_Arms = torch.tensor([df_output_ML_Both_Arms])
            
            
            if row_no_start == 0 and file_no == 1:
                input_tensor = current_input
                output_Locomotion_tensor = current_output_Locomotion
                output_ML_Both_Arms_tensor = current_output_ML_Both_Arms
            else:
                input_tensor = torch.cat((input_tensor, current_input), 0)
                output_Locomotion_tensor = torch.cat((output_Locomotion_tensor, current_output_Locomotion), 0)
                output_ML_Both_Arms_tensor = torch.cat((output_ML_Both_Arms_tensor, current_output_ML_Both_Arms), 0)
                
            row_no_start += SILDE_WINDOW_STEP
            #print(input_tensor.size())
            #print(input_tensor)
    return input_tensor, output_Locomotion_tensor, output_ML_Both_Arms_tensor


"""
Training_x, Training_y_1, Training_y_2 = data_generator(df_OPPORTUNITY_DATASET_Training_WIP, SILDE_WINDOW, SILDE_WINDOW_STEP, OPPORTUNITY_column_name_selected)
Training_x = torch.reshape(Training_x, (Training_x.size()[0], 1, Training_x.size()[1], Training_x.size()[2])).float()
"""
"""
Testing_x, Testing_y_1, Testing_y_2 = data_generator(df_OPPORTUNITY_DATASET_Testing_WIP, SILDE_WINDOW, SILDE_WINDOW_STEP, OPPORTUNITY_column_name_selected)
Testing_x = torch.reshape(Testing_x, (Testing_x.size()[0], 1, Testing_x.size()[1], Testing_x.size()[2])).float()
"""
Validation_x, Validation_y_1, Validation_y_2 = data_generator(df_OPPORTUNITY_DATASET_Validation_WIP, SILDE_WINDOW, SILDE_WINDOW_STEP, OPPORTUNITY_column_name_selected)
Validation_x = torch.reshape(Validation_x, (Validation_x.size()[0], 1, Validation_x.size()[1], Validation_x.size()[2])).float()

#def batch_generator():


1
2


In [23]:
Validation_x.size()

torch.Size([5076, 1, 24, 113])

In [24]:
df_OPPORTUNITY_DATASET_Validation_WIP[OPPORTUNITY_column_name_selected[1: -3]][1: 2]

,2 Accelerometer RKN^ accX,3 Accelerometer RKN^ accY,4 Accelerometer RKN^ accZ,5 Accelerometer HIP accX,6 Accelerometer HIP accY,7 Accelerometer HIP accZ,8 Accelerometer LUA^ accX,9 Accelerometer LUA^ accY,10 Accelerometer LUA^ accZ,11 Accelerometer RUA_ accX,...,125 InertialMeasurementUnit R-SHOE Body_Ax,126 InertialMeasurementUnit R-SHOE Body_Ay,127 InertialMeasurementUnit R-SHOE Body_Az,128 InertialMeasurementUnit R-SHOE AngVelBodyFrameX,129 InertialMeasurementUnit R-SHOE AngVelBodyFrameY,130 InertialMeasurementUnit R-SHOE AngVelBodyFrameZ,131 InertialMeasurementUnit R-SHOE AngVelNavFrameX,132 InertialMeasurementUnit R-SHOE AngVelNavFrameY,133 InertialMeasurementUnit R-SHOE AngVelNavFrameZ,134 InertialMeasurementUnit R-SHOE Compass
1,0.488167,0.674167,0.488833,0.4905,0.676167,0.486,0.517333,0.658333,0.550833,0.61,...,0.4985,0.5508,0.4235,0.4997,0.50215,0.50195,0.50215,0.5003,0.50195,0.78


In [25]:
Validation_y_2

tensor([0, 0, 0,  ..., 0, 0, 0])

In [26]:
df_OPPORTUNITY_DATASET_Validation_WIP['250 ML_Both_Arms'].unique()
#df_OPPORTUNITY_DATASET_Validation_WIP['244 Locomotion'].unique()

array([ 0, 12,  4,  3,  7,  8,  9, 10,  5,  6, 15, 16,  1,  2, 11, 13, 14,
       17], dtype=int64)

In [27]:
#df_OPPORTUNITY_DATASET_WIP['22 Accelerometer RKN_ accZ'].unique()

pending items:
    - batch size = 100
    - sensor data were pre-processed to fill in missing values using linear interpolation
    - model accuracy & loss function save into array
    - Dashboard - to show performance
    
    - per channel normalization to interval [0,1]
    https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame

In [43]:
"""
input = np.array([random() for i in range(24 * 113 * 1 * 6)]).reshape(6, 1, 24, 113)

output = torch.Tensor([[1.0, 0.0, 0.0, 0.0, 0.0],
                     [0.0, 1.0, 0.0, 0.0, 0.0],
                     [0.0, 0.0, 0.0, 1.0, 0.0],
                     [0.0, 0.0, 1.0, 0.0, 0.0],
                     [0.0, 0.0, 1.0, 0.0, 0.0],
                     [0.0, 0.0, 0.0, 0.0, 1.0]])


output = torch.Tensor([0, 1, 3, 2, 2, 4])

final_input = torch.Tensor(input)
final_output = torch.Tensor(output)
"""

class CNN_LSTM_HAR_MODEL(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn2d_1 = nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = (5, 1))
        self.relu = nn.ReLU()
        self.cnn2d_2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (5, 1))
        self.cnn2d_3 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (5, 1))
        self.cnn2d_4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = (5, 1))
        
        self.batch_size = 24 - 4 * (5 - 1)
        self.hidden_dim = 128
        self.input_size = 7232
        self.n_layers = 1
        self.lstm1 = nn.LSTM(input_size  = 7232, hidden_size = self.hidden_dim, num_layers = self.n_layers, dropout = 0.5)
        
        self.batch_size = 24 - 4 * (5 - 1)
        self.hidden_dim = 128
        self.input_size = 128
        self.n_layers = 1
        self.lstm2 = nn.LSTM(input_size  = self.input_size, hidden_size = self.hidden_dim, num_layers = self.n_layers, dropout = 0.5)
        
        self.fc1 = nn.Linear(8 * 128, 8 * 18)
        
    def forward(self, x):
        #print(x)
        x = self.cnn2d_1(x)
        x = self.relu(x)
        #print(x)
        x = self.cnn2d_2(x)
        x = self.relu(x)
        #print(x)
        x = self.cnn2d_3(x)
        x = self.relu(x)
        #print(x)
        x = self.cnn2d_4(x)
        x = self.relu(x)
        #print(x)
        x = x.transpose(1, 2)
        #print(x)
        x = torch.reshape(x, (x.size()[0], x.size()[1], -1))
        #print(x)
        hidden = self.init_hidden()
        x, hidden = self.lstm1(x, hidden)
        #print(x)
        x, hidden = self.lstm2(x, hidden)
        #print(x)
        x = torch.reshape(x, (x.size()[0], -1))
        #print(x)
        x = self.fc1(x)
        #print(x)
        x = F.softmax(x)
        
        x = torch.reshape(x, (x.size()[0], 8, 18))
        x = x[:, -1, :]
        #print(x)
        return x
    
    def init_hidden(self):
        weight = next(self.parameters()).data

        hidden = (weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, self.batch_size, self.hidden_dim).zero_())
        return hidden

    
    
    
    
    




In [29]:
list(Validation_y_2.size())[0]

5076

In [ ]:
BATCH_SIZE = 1500


CNN_LSTM_HAR_MODEL_WIP = CNN_LSTM_HAR_MODEL()
learning_rate = 0.001
#learning_rate = 0.02
loss_functioin = nn.CrossEntropyLoss(weight = 8.0 * torch.tensor([0.2/0.8/17, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]))
#optimizer = optim.SGD(CNN_LSTM_HAR_MODEL_WIP.parameters(), lr = learning_rate)
#https://pytorch.org/docs/stable/optim.html
optimizer = torch.optim.RMSprop(CNN_LSTM_HAR_MODEL_WIP.parameters(), lr = learning_rate, weight_decay = 0.9)



Training_set = 'Validation'




epoch_size = 20
steps_for_printing_out_loss = 1

data_size = list(Validation_y_2.size())[0]

for i in range(1, epoch_size + 1):
    optimizer.zero_grad()
    no_of_right = 0
    epoch_loss = 0
    for Batch_no in range(int(data_size / BATCH_SIZE) + 1):
        #print('Batch_no: ' + str(Batch_no))
        batch_start = Batch_no * BATCH_SIZE
        batch_end = min(data_size, (Batch_no + 1) * BATCH_SIZE)

        if Training_set == 'Validation':
            input = Validation_x[batch_start: batch_end]
            target = Validation_y_2[batch_start: batch_end]
        elif Training_set == 'Testing':
            input = Testing_x[batch_start: batch_end]
            target = Testing_y_2[batch_start: batch_end]
        elif Training_set == 'Training':
            input = Training_x[batch_start: batch_end]
            target = Training_y_2[batch_start: batch_end]
        
        output = CNN_LSTM_HAR_MODEL_WIP(input)
        loss = loss_functioin(output, target.long())
        epoch_loss += loss
        loss.backward()
        if i == 1 or i % (steps_for_printing_out_loss) == 0:
            #print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
            
            for k in range(len(target)):
                if output.argmax(1)[k] == target[k]:
                    no_of_right += 1
    if i == 1 or i % (steps_for_printing_out_loss) == 0:
        print("Epoch {}/{}, Loss: {:.6f}, Accuracy: {:.6f}".format(i, epoch_size, epoch_loss.cpu().detach().numpy(), no_of_right / data_size))
    optimizer.step()

<ipython-input-43-9ce7ccd517f3>:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


Epoch 1/20, Loss: 11.561323, Accuracy: 0.009062
Epoch 2/20, Loss: 11.561319, Accuracy: 0.009259
Epoch 3/20, Loss: 11.561296, Accuracy: 0.009850
Epoch 4/20, Loss: 11.561274, Accuracy: 0.009850
Epoch 5/20, Loss: 11.561285, Accuracy: 0.009850
Epoch 6/20, Loss: 11.561283, Accuracy: 0.009850
Epoch 7/20, Loss: 11.561286, Accuracy: 0.039204
Epoch 8/20, Loss: 11.561288, Accuracy: 0.039204
Epoch 9/20, Loss: 11.561293, Accuracy: 0.039204
Epoch 10/20, Loss: 11.561287, Accuracy: 0.039204
Epoch 11/20, Loss: 11.561283, Accuracy: 0.039204
Epoch 12/20, Loss: 11.561283, Accuracy: 0.039204
Epoch 13/20, Loss: 11.561286, Accuracy: 0.039204
Epoch 14/20, Loss: 11.561286, Accuracy: 0.039204


In [38]:
def tensor_to_csv(tensor_name):
    x_np = tensor_name.numpy()
    x_df = pd.DataFrame(x_np)
    x_df.to_csv('tmp.csv')

Validation_x.size()
output

tensor_to_csv(output.detach())


#Validation_x_to_csv = torch.reshape(Validation_x, (Validation_x.size()[0] * Validation_x.size()[1] * Validation_x.size()[2], Validation_x.size()[3])).float()

#tensor_to_csv(Validation_x[0, 0,:, :])

#tensor_to_csv(Validation_y_2)

for row_no_end in range(0, 155):
    print(row_no_end)
    print(df_OPPORTUNITY_DATASET_Validation_WIP['250 ML_Both_Arms'][row_no_end])

0
0    0
0    0
Name: 250 ML_Both_Arms, dtype: int64
1
1    0
1    0
Name: 250 ML_Both_Arms, dtype: int64
2
2    0
2    0
Name: 250 ML_Both_Arms, dtype: int64
3
3    0
3    0
Name: 250 ML_Both_Arms, dtype: int64
4
4    0
4    0
Name: 250 ML_Both_Arms, dtype: int64
5
5    0
5    0
Name: 250 ML_Both_Arms, dtype: int64
6
6    0
6    0
Name: 250 ML_Both_Arms, dtype: int64
7
7    0
7    0
Name: 250 ML_Both_Arms, dtype: int64
8
8    0
8    0
Name: 250 ML_Both_Arms, dtype: int64
9
9    0
9    0
Name: 250 ML_Both_Arms, dtype: int64
10
10    0
10    0
Name: 250 ML_Both_Arms, dtype: int64
11
11    0
11    0
Name: 250 ML_Both_Arms, dtype: int64
12
12    0
12    0
Name: 250 ML_Both_Arms, dtype: int64
13
13    0
13    0
Name: 250 ML_Both_Arms, dtype: int64
14
14    0
14    0
Name: 250 ML_Both_Arms, dtype: int64
15
15    0
15    0
Name: 250 ML_Both_Arms, dtype: int64
16
16    0
16    0
Name: 250 ML_Both_Arms, dtype: int64
17
17    0
17    0
Name: 250 ML_Both_Arms, dtype: int64
18
18    0
18    0
Nam

In [ ]:
input.size()
target
CNN_LSTM_HAR_MODEL_WIP.parameters()
target
Validation_y_2.sum()
print(Validation_y_2)
for i in Validation_y_2.long():
    print(i)

In [ ]:
for i in CNN_LSTM_HAR_MODEL_WIP.parameters():
    print(i.size())
    print(i)

In [ ]:

torch.isnan(Validation_x)



x_np = Validation_x[0, 0,:, :].numpy()
x_df = pd.DataFrame(x_np)
x_df.to_csv('tmp.csv')





In [ ]:
>>> rnn = nn.LSTM(10, 20, 2)
>>> input = torch.randn(5, 3, 10)
>>> h0 = torch.randn(2, 3, 20)
>>> c0 = torch.randn(2, 3, 20)
>>> output, (hn, cn) = rnn(input, (h0, c0))

output.size()
#hn.size()

In [ ]:
import scipy.io
mat = scipy.io.loadmat('Dataset/SKODA_DATASET/SkodaMiniCP_2015_08/left_classall_clean.mat')
mat

In [ ]:
kk = torch.tensor([[[k* 100 + j * 10 + i for i in range (10)] for j in range(10)] for k in range(10)])
kk
mm = torch.reshape(kk, (kk.size()[0], 1, kk.size()[1], kk.size()[2])).float()
mm.size()
mm[:, 0, :, :] == kk
mm[5, 0, 4, 3]